# Population Data
This notebook pulls data from Wikipedia with the goal of createing a dataframe with population data in each county in the US

In [2]:
#!pip install wikipedia

In [3]:
#import wikipedia

ModuleNotFoundError: No module named 'wikipedia'

# No path to the wikipedia libary
Since my machine seems to be having issues with its path, I'll pull this data using beautiful soup and stick to what is working as a time saver.

Import requests to access the web and BeautifulSoup to parse it


In [80]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Create a variable that leads to the main wiki page that we want to scrape. In this case, we want population data for every county in the United States. We also want FIPS data if we can find it.

In [5]:
url = 'https://en.wikipedia.org/wiki/County_statistics_of_the_United_States'
res = requests.get(url)
res.status_code

200

In [ ]:
big_soup_bowl = BeautifulSoup(res.content, 'lxml')     # create a BS object for the main page using 'lxml'

Use "find_all" to access the table on this page that holds the data that we want, here the states

In [100]:
table = big_soup_bowl.find_all("tr")
table[1].text.split("\n")

['', '254', ' Texas', 'The most counties of any U.S. state', '']

In [72]:
table[1].find_all("a", href=True)

[<a href="/wiki/Texas" title="Texas"><img alt="Texas" class="thumbborder" data-file-height="720" data-file-width="1080" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/f7/Flag_of_Texas.svg/23px-Flag_of_Texas.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/f7/Flag_of_Texas.svg/35px-Flag_of_Texas.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/f7/Flag_of_Texas.svg/45px-Flag_of_Texas.svg.png 2x" width="23"/></a>,
 <a href="/wiki/List_of_counties_in_Texas" title="List of counties in Texas">Texas</a>]

In [101]:
table[1].text.split("\n")[1]

'254'

In [103]:
table[1].text.split("\n")[2].strip()

'Texas'

In [104]:
'https://en.wikipedia.org/wiki/List_of_counties_in_' + table[1].text.split("\n")[2].strip()

'https://en.wikipedia.org/wiki/List_of_counties_in_Texas'

I want to create a dictionary where the key is the state or territory name and the value is a link to the wiki page that holds the populations for the counties and any other data that might be relevant.  BeautifulSoup can do that.

In [176]:
states = {}

name = []
num_county = []
link = []
flag = False

for i in range(1, 60):
    if not flag:
        num_county.append(int(table[i].text.split("\n")[1]))
        name.append(str(table[i].text.split("\n")[2].strip()))
        link.append(str('https://en.wikipedia.org/wiki/List_of_counties_in_' + table[i].text.split("\n")[2].strip()))
        if str(table[i].text.split("\n")[2].strip()) == 'Guam':
            flag=True            
        
states['state_name'] = name
states['num_counties'] = num_county
states['link'] = link
states

{'state_name': ['Texas',
  'Georgia',
  'Virginia',
  'Kentucky',
  'Missouri',
  'Kansas',
  'Illinois',
  'North Carolina',
  'Iowa',
  'Tennessee',
  'Nebraska',
  'Indiana',
  'Ohio',
  'Minnesota',
  'Michigan',
  'Mississippi',
  'Puerto Rico',
  'Oklahoma',
  'Arkansas',
  'Wisconsin',
  'Alabama',
  'Florida',
  'Pennsylvania',
  'South Dakota',
  'Colorado',
  'Louisiana',
  'New York',
  'California',
  'Montana',
  'West Virginia',
  'North Dakota',
  'South Carolina',
  'Idaho',
  'Washington',
  'Oregon',
  'New Mexico',
  'Alaska',
  'Utah',
  'Maryland',
  'Wyoming',
  'New Jersey',
  'Nevada',
  'Maine',
  'Arizona',
  'Massachusetts',
  'Vermont',
  'New Hampshire',
  'U.S. Minor Outlying Islands',
  'Connecticut',
  'American Samoa',
  'Hawaii',
  'Rhode Island',
  'Northern Mariana Islands',
  'Delaware',
  'Virgin Islands (U.S.)',
  'District of Columbia',
  'Guam'],
 'num_counties': [254,
  159,
  133,
  120,
  115,
  105,
  102,
  100,
  99,
  95,
  93,
  92,
  88

In [177]:
state_df = pd.DataFrame(states)
state_df.head()

,state_name,num_counties,link
0,Texas,254,https://en.wikipedia.org/wiki/List_of_counties...
1,Georgia,159,https://en.wikipedia.org/wiki/List_of_counties...
2,Virginia,133,https://en.wikipedia.org/wiki/List_of_counties...
3,Kentucky,120,https://en.wikipedia.org/wiki/List_of_counties...
4,Missouri,115,https://en.wikipedia.org/wiki/List_of_counties...


In [178]:
state_df.dtypes

state_name      object
num_counties     int64
link            object
dtype: object

In [179]:
states['link'][0]

'https://en.wikipedia.org/wiki/List_of_counties_in_Texas'

In [217]:
def get_county_info(num):
    url = str(states['link'][num])
    res = requests.get(url)
    if res.status_code == 200:
        cuppa_soup = BeautifulSoup(res.content, 'lxml')
        state_table = cuppa_soup.find_all("table")
    else:
        print('There was an error.  The code is', res.status_code)
    return state_table

In [220]:
texas = get_county_info(0)

In [221]:
header = texas[1].find("tr").text.split("\n")
for head in header:
    if len(head) < 2:
        header.remove(head)

header

['County',
 'FIPS code[8]',
 'County seat[9]',
 'Est.[9]',
 'Origin',
 'Etymology',
 'Population[10]',
 'Area[9]',
 'Map',
 '']

In [222]:
georgia = get_county_info(1)

In [228]:
georgia[1].find("tr")

<tr><td class="mbox-image"><div style="width:52px"><a class="image" href="/wiki/File:Question_book-new.svg"><img alt="" data-file-height="399" data-file-width="512" decoding="async" height="39" src="//upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/50px-Question_book-new.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/75px-Question_book-new.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/100px-Question_book-new.svg.png 2x" width="50"/></a></div></td><td class="mbox-text"><div class="mbox-text-span">This section <b>needs additional citations for <a href="/wiki/Wikipedia:Verifiability" title="Wikipedia:Verifiability">verification</a></b>.<span class="hide-when-compact"> Please help <a class="external text" href="https://en.wikipedia.org/w/index.php?title=List_of_counties_in_Georgia&amp;action=edit">improve this article</a> by <a href="/wiki/Help:Referencing_for_beginners" title="Help:Referencing f

In [216]:
header.insert(0, 'state')
header

['state',
 'This section needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed. (December 2015) (Learn how and when to remove this template message)']

In [187]:
header = ['state_name', 'county_name', 'FIPS_code', 'population', 'area']

In [188]:
state_name_cnty = []
county_name = []
fips_code = []
population = []
area = []
counties = {'state_name': state_name_cnty, 'county_name': county_name, 
            'FIPS_code': fips_code, 'population': population, 'area': area}

In [190]:
texas[1].find_all("tr")[1].text.split("\n")

['',
 'Anderson County',
 '',
 '001',
 '',
 'Palestine',
 '1846',
 'Houston County',
 'Kenneth Lewis Anderson (1805–1845), the last vice president of the Republic of Texas',
 '',
 '57,741',
 '',
 '1,071\xa0sq\xa0mi(2,774\xa0km2)',
 '',
 '']

In [205]:
# demonstrate each entry

cnty = texas[1].find_all("tr")[1].text.split("\n")[1]
print(cnty)
fips = int(texas[1].find_all("tr")[1].text.split("\n")[3])
print(fips)
pop = int(texas[1].find_all("tr")[1].text.split("\n")[10].replace(',', ''))
print(pop)
ar = int(texas[1].find_all("tr")[1].text.split("\n")[10].replace(',',''))
print(ar)

In [180]:
states['num_counties'][0]

254

In [208]:
states['state_name'][0]

'Texas'

In [174]:
states.keys()

dict_keys(['state_name', 'num_counties', 'link'])

In [181]:
len(states)

3

### Create a dictionary of the counties with the data scraped from their pages
Turns out this won't work because each state's wiki county page is structured differently. This structure that we careated above only works for Texas

In [212]:
count=0
for link in states['link']:
    # use the link that we are in to access the state's county page in wikipedia
    url = str(link)
    res = requests.get(url)
    try:
        # create a beautiful soup object
        cuppa_soup = BeautifulSoup(res.content, 'lxml')
        tabs = cuppa_soup.find_all("table")
        suc = states['state_name'][count]
        print(f'successfully accessed {suc}\'s county data')
        count+=1
    except:
        print('There was an error.  The code is', res.status_code)
        err = states['state_name'][count]
        print(f'the state of error is {err}')
        count+=1
        
    finally:
        print('fially folks ......')
    
print(count)
    

successfully accessed Texas's county data
fially folks ......
successfully accessed Georgia's county data
fially folks ......
successfully accessed Virginia's county data
fially folks ......
successfully accessed Kentucky's county data
fially folks ......
successfully accessed Missouri's county data
fially folks ......
successfully accessed Kansas's county data
fially folks ......
successfully accessed Illinois's county data
fially folks ......
successfully accessed North Carolina's county data
fially folks ......
successfully accessed Iowa's county data
fially folks ......
successfully accessed Tennessee's county data
fially folks ......
successfully accessed Nebraska's county data
fially folks ......
successfully accessed Indiana's county data
fially folks ......
successfully accessed Ohio's county data
fially folks ......
successfully accessed Minnesota's county data
fially folks ......
successfully accessed Michigan's county data
fially folks ......
successfully accessed Mississipp